In [4]:
import time
import numpy as np
from math import sin, cos, atan2
from coppeliasim_zmqremoteapi_client import RemoteAPIClient

MAC = '''
ROBOT_NAME = "Pioneer_p3dx" # MAC
LEFT_MOTOR_NAME = ROBOT_NAME + "_leftMotor" # MAC
RIGHT_MOTOR_NAME = ROBOT_NAME + "_rightMotor" # MAC
SENSOR_PATTERN = "Pioneer_p3dx_ultrasonicSensor{}" # MAC
'''
#LINUX = '''
ROBOT_NAME = "PioneerP3DX" # Linux
LEFT_MOTOR_NAME = "/PioneerP3DX/leftMotor" # Linux
RIGHT_MOTOR_NAME = "/PioneerP3DX/rightMotor" #L inux
SENSOR_PATTERN = "/PioneerP3DX/ultrasonicSensor[{}]" # Linux
#'''

L = 0.381      # distância entre rodas 
r = 0.0975     # raio da roda

# parametros para calibrar
K_att = 1.0
K_rep = 0.8
kv = 1.5
kw = 3.5

# limites e parâmetros do comportamento
d0 = 1.0 # raio da repulsão
v_max = 0.6
w_max = 1.5

# condição de parada
DIST_GOAL_STOP = 0.3 

In [5]:
def angle_diff(a, b):
    """Retorna diferença angular (a - b) normalizada para [-pi, pi]."""
    d = a - b
    return np.arctan2(np.sin(d), np.cos(d))

def clip(x, a, b):
    return max(a, min(b, x))

def transform_point_matrix(sim, mat4, point3):

    M = np.array(mat4).reshape((3,4))
    p = np.array([point3[0], point3[1], point3[2], 1.0])
    res = M @ p
    return res  # 3-vector



In [ ]:
try:
    client = RemoteAPIClient()
    sim = client.require("sim")
    sim.setStepping(True)
    
    # Frame que representa o Goal
    goal = 'Goal'
    goalFrame = sim.getObject('/' + goal)
    # Posição do Goal (x, y, theta)
    qgoal = sim.getObjectPosition(goalFrame, sim.handle_world)
    qgoal_orientation = sim.getObjectOrientation(goalFrame, sim.handle_world)

    # Handles do robô e rodas

    MAC = '''
    robotHandle = sim.getObject('Pioneer_p3dx') # MAC
    l_wheel = sim.getObject('Pioneer_p3dx_leftMotor') # MAC
    r_wheel = sim.getObject('Pioneer_p3dx_rightMotor') # MAC
    sensorHandles = [sim.getObject(f'Pioneer_p3dx_ultrasonicSensor{i}') for i in range(1, 17)] # MAC
    '''
    #LINUX = '''
    robotHandle = sim.getObject('/PioneerP3DX') # Linux
    l_wheel = sim.getObject('/PioneerP3DX/leftMotor') # Linux
    r_wheel = sim.getObject('/PioneerP3DX/rightMotor') # Linux
    sensorHandles = [sim.getObject(f'/PioneerP3DX/ultrasonicSensor[{i}]') for i in range(16)] # Linux
    #'''
    
    if len(sensorHandles) == 0:
        raise Exception("AVISO: nenhum sensor encontrado. Verifique nomes na cena.")
    



except Exception as e:
    print(f"An error occurred: {e}")